In [ ]:
!pip install beautifulsoup4
!pip install pyopenssl ndg-httpsclient pyasn1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 776 kB/s 
     |████████████████████████████████| 4.1 MB 3.5 MB/s 


In [ ]:
!pip install thread6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import urllib.request as openurl
import urllib
import ssl
#ssl._create_default_https_context = ssl._create_unverified_context
from bs4 import BeautifulSoup
import pandas as pd
import time
from random import randint
import requests
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
disable_warnings(InsecureRequestWarning)
import threading

# Multithread

In [ ]:
# code fix
def get_tickers():
  html_link = 'https://hnx.vn/vi-vn/cophieu-etfs/chung-khoan-uc.html'
  s = requests.get(html_link, verify = False).text
  soup = BeautifulSoup(s, 'html.parser')
  data = pd.DataFrame( columns = ['ticker','company'])
  for option in soup.find_all('option'):
    data_ticker = pd.DataFrame([{'ticker': option['value'], 'company': option.text}])
    data = pd.concat([data,data_ticker],ignore_index=True)
  tickers = data.ticker[:-5]
  return tickers

def data_tickers(ticker):
  global data_content
  data_content = pd.DataFrame(columns = ['ticker','title','content'])
  sticker_test = 'https://hnx.vn/cophieu-etfs/chi-tiet-chung-khoan-uc-%s.html?_des_tab=1' %ticker
  print(ticker)
  #with openurl.urlopen(sticker_test,timeout=1000) as url:
  #  r = url.read()
  r = requests.get(sticker_test,verify = False).text
  company_data = BeautifulSoup(r, 'html.parser')
  data_raw_v2 = zip(company_data.find_all("div", {"class": "dktimkiem_cell_title"}),company_data.find_all("div", {"class": "dktimkiem_cell_content"}))
  for titles, contents in data_raw_v2:
    data_content_raw = pd.DataFrame([{'ticker':ticker,'title':titles.text,'content':contents.text}] )
    data_content = pd.concat([data_content,data_content_raw],ignore_index = True)
  openurl.urlcleanup()
  print('Done',ticker)
  timesleep = randint(1,20)
  print('sleeping time',timesleep)
  time.sleep(timesleep)
  data_content = data_content.replace(r'\n',' ', regex=True) 
  data_content = data_content.replace(r'\r',' ', regex=True) 
  return data_content

In [ ]:
tickers = get_tickers()
global data_content
data_raw = pd.DataFrame(columns = ['ticker','title','content'])
start = time.perf_counter()
threads = []
for ticker in tickers:
    t = threading.Thread(target=data_tickers, args=[ticker])
    t.start()
    threads.append(t)
    data_raw = pd.concat([data_raw,data_content], ignore_index = True)
for thread in threads:
    thread.join()
finish = time.perf_counter()
print(f'Finished in {round(finish-start, 2)} seconds') 

SSLError: ignored